<a href="https://colab.research.google.com/github/martinkenas/Voice-controlled-conversation-robot_Lithuanian_Language/blob/main/SpeechRobot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# 1. Įdiekite reikiamas bibliotekas
!pip install google-cloud-speech
!pip install google-cloud-texttospeech
!pip install pydub
!pip install google-cloud-dialogflow

**SPEECH TO TEXT**   "Nauseda.flac" is an audio file derived from this video https://www.youtube.com/watch?v=XZJM4QMWT9U

In [8]:
# 2. Importuokite bibliotekas
from google.cloud import speech
import io
import os
from pydub import AudioSegment

# 3. Nustatykite savo GCP projekto ID ir API raktą
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "API KEY JSON"

# 4. Sukurkite Speech-to-Text klientą
client = speech.SpeechClient()

# 5. Nurodykite garso failo kelią (pakeiskite PATH_TO_YOUR_AUDIO_FILE)
file_name = "/content/Nauseda.flac"

# 6. Atidarykite ir konvertuokite garso failą į mono
audio_file = AudioSegment.from_file(file_name)
audio_file = audio_file.set_channels(1)  # Konvertuokite į mono

# 7. Išsaugokite konvertuotą failą
mono_file_name = "/content/Nauseda_mono.flac"
audio_file.export(mono_file_name, format="flac")

# 8. Atidarykite konvertuotą garso failą
with io.open(mono_file_name, "rb") as f:
    content = f.read()

# 9. Sukurkite Speech-to-Text užklausą
audio = speech.RecognitionAudio(content=content)
config = speech.RecognitionConfig(
    encoding=speech.RecognitionConfig.AudioEncoding.FLAC,
    sample_rate_hertz=44100,
    language_code="lt-LT",  # Lietuvių kalba
)

# 10. Atsiųskite užklausą ir gaukite atsakymą
try:
    response = client.recognize(config=config, audio=audio)

    if not response.results:
        print("No speech recognized")
    for result in response.results:
        print("Transcript: {}".format(result.alternatives[0].transcript))
except Exception as e:
    print(f"Error occurred: {e}")


Transcript: Vokietijos kariuomenė žinote nėra mergina kurią galima pakviesti gerai praleisti vakaro prie ežero patvirtintų Ne tai yra rimta kariuomenė Kuriai reikia pasiūlyti vedybų sutartį ir pavedimu sutartyje turi būti labai neaiški numatyti Lietuvos įsipareigojimai ką mes padarysime Nes kitaip Lietuvos Kinijos kariuomenė savo įsipareigojimo mums taip pat negalėsiu eiti


**TEXT TO SPEECH**

In [9]:
from google.cloud import texttospeech

def synthesize_text(text):
    client = texttospeech.TextToSpeechClient()
    input_text = texttospeech.SynthesisInput(text=text)

    voice = texttospeech.VoiceSelectionParams(
        language_code="lt-LT",
        ssml_gender=texttospeech.SsmlVoiceGender.NEUTRAL
    )

    audio_config = texttospeech.AudioConfig(
        audio_encoding=texttospeech.AudioEncoding.MP3
    )

    response = client.synthesize_speech(
        input=input_text, voice=voice, audio_config=audio_config
    )

    with open("Pasisveikinimas.mp3", "wb") as out:
        out.write(response.audio_content)
        print("Audio content written to file 'Pasisveikinimas.mp3'")


In [10]:
text = "Sveiki, kaip jums sekasi?"
synthesize_text(text)


Audio content written to file 'Pasisveikinimas.mp3'


**Sending Voice Request and Receiving Voice Answer From Dialogflow Bot**

In [14]:
# 2. Importuokite bibliotekas
from google.cloud import speech
from google.cloud import texttospeech
from google.cloud import dialogflow_v2 as dialogflow
import io
import os
from pydub import AudioSegment

# 3. Nustatykite savo GCP projekto ID ir API raktą
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "API KEY JSON"

# 4. Sukurkite klientus
speech_client = speech.SpeechClient()
text_to_speech_client = texttospeech.TextToSpeechClient()
dialogflow_client = dialogflow.SessionsClient()

# 5. Nurodykite garso failo kelią (pakeiskite PATH_TO_YOUR_AUDIO_FILE)
audio_file_path = "/content/Pasisveikinimas.flac"

# 6. Funkcija konvertuoti garso failą į mono
def convert_to_mono(file_path):
    audio = AudioSegment.from_file(file_path)
    audio = audio.set_channels(1)
    mono_file_path = "/content/mono_audio_Pasisveikinimas.flac"
    audio.export(mono_file_path, format="flac")
    return mono_file_path

# 7. Funkcija atpažinti kalbą iš garso failo
def recognize_speech(file_path):
    mono_file_path = convert_to_mono(file_path)
    with io.open(mono_file_path, "rb") as f:
        content = f.read()
    audio = speech.RecognitionAudio(content=content)
    config = speech.RecognitionConfig(
        encoding=speech.RecognitionConfig.AudioEncoding.FLAC,
        sample_rate_hertz=24000,
        language_code="lt-LT",
    )
    response = speech_client.recognize(config=config, audio=audio)
    for result in response.results:
        return result.alternatives[0].transcript
    return "No speech recognized"

# 8. Funkcija siųsti užklausą į Dialogflow ir gauti atsakymą
def detect_intent_texts(project_id, session_id, text, language_code):
    session = dialogflow_client.session_path(project_id, session_id)
    text_input = dialogflow.TextInput(text=text, language_code=language_code)
    query_input = dialogflow.QueryInput(text=text_input)
    response = dialogflow_client.detect_intent(
        request={"session": session, "query_input": query_input}
    )
    return response.query_result.fulfillment_text

# 9. Funkcija konvertuoti tekstą į kalbą
def synthesize_text(text):
    input_text = texttospeech.SynthesisInput(text=text)
    voice = texttospeech.VoiceSelectionParams(
        language_code="lt-LT",
        ssml_gender=texttospeech.SsmlVoiceGender.NEUTRAL
    )
    audio_config = texttospeech.AudioConfig(audio_encoding=texttospeech.AudioEncoding.MP3)
    response = text_to_speech_client.synthesize_speech(
        input=input_text, voice=voice, audio_config=audio_config
    )
    with open("Pasisveikinimas_ATSAKYMAS.mp3", "wb") as out:
        out.write(response.audio_content)
        print("Audio content written to file 'Pasisveikinimas_ATSAKYMAS.mp3'")

# 10. Pagrindinė funkcija apjungiant visus žingsnius
def main(audio_file_path):
    # Atpažinkite kalbą iš garso failo
    recognized_text = recognize_speech(audio_file_path)
    print(f"Recognized text: {recognized_text}")

    # Siųskite užklausą į Dialogflow ir gaukite atsakymą
    project_id = "speech-control-robot"
    session_id = "52b225d7-d972-4a7b-8226-4697b8029731"
    response_text = detect_intent_texts(project_id, session_id, recognized_text, "lt")
    print(f"Dialogflow response: {response_text}")

    # Konvertuokite atsakymą į kalbą
    synthesize_text(response_text)

# Pavyzdinis naudojimas
main(audio_file_path)


Recognized text: Sveiki kaip jum sekasi
Dialogflow response: Sveiki atvykę į mūsų žaidimų parduotuvę.  Kuom galiu padėti?
Audio content written to file 'Pasisveikinimas_ATSAKYMAS.mp3'
